<a href="https://colab.research.google.com/github/LeninGF/CoursesNotes/blob/main/InteligenciaArtificalGenerativa/Problems/transformersII-huggingface/Ejercicios-IAG-2024B-LeninFalconi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Ejericios Transformers con HuggingFace

Coder: Lenin Falconí

Fecha: 2024-12-16



## Resumen de Texto


In [4]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [6]:
from datasets import load_dataset

dataset = load_dataset("ILSUM/ILSUM-1.0", "English")
print(f"Features: {dataset['train'].column_names}")

README.md:   0%|          | 0.00/4.78k [00:00<?, ?B/s]

train.csv:   0%|          | 0.00/46.5M [00:00<?, ?B/s]

test.csv:   0%|          | 0.00/16.8M [00:00<?, ?B/s]

val.csv:   0%|          | 0.00/3.37M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/12565 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/4487 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/898 [00:00<?, ? examples/s]

Features: ['id', 'Article', 'Heading', 'Summary']


In [8]:
example = dataset["train"][21]
example["Article"]

'This is how an Apple Watch saved a man\'s life after detecting accidentIt all started when Gabe Burdett was waiting for his father Bob at their pre-designated location for some mountain biking at the Riverside State Park when he received a text alert from his dad\'s Apple Watch, saying it had detected a "hard fall".Burdett, from city of Spokane in Washington State later received another update from the Watch, saying his father had reached Sacred Heart Medical Center."We drove straight there but he was gone when we arrived. I get another update from the Watch saying his location has changed with a map location of SHMC. Dad flipped his bike at the bottom of Doomsday, hit his head and was knocked out until sometime during the ambulance ride," Burdett wrote in a Facebook post.The Watch notified 911 with the location and within 30 minutes, emergency medical services (EMS) took the injured Bob to the hospital."If you own an Apple Watch, set up your hard fall detection, it\'s not just for wh

In [9]:
example["Summary"]

'“Dad flipped his bike at the bottom of Doomsday, hit his head and was knocked out until sometime during the ambulance ride. The watch had called 911 with his location and EMS had him scooped up and to the hospital in under a 1/2 hr,” says Gabe in his post.'

In [11]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
model_name = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
input_ids = tokenizer.encode("summarize: " + example["Article"],
return_tensors="pt",
max_length=512,
truncation=True
 )

In [13]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
model_name = "Helsinki-NLP/opus-mt-en-es"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

english_inputs = ["Hello, it was raining when I arrived today.", "what is your name?", "There was a time when Green Leaves was a popular folk song"]
for english_input in english_inputs:
  input_ids = tokenizer(english_input, return_tensors="pt")
  translated_ids = model.generate(**input_ids)
  translated_text = tokenizer.decode(translated_ids[0], skip_special_tokens=True)
  print(f"English: {english_input}: Spanish: {translated_text}")

tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/826k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.59M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


pytorch_model.bin:   0%|          | 0.00/312M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

English: Hello, it was raining when I arrived today.: Spanish: Hola, estaba lloviendo cuando llegué hoy.
English: what is your name?: Spanish: ¿Cómo te llamas?
English: There was a time when Green Leaves was a popular folk song: Spanish: Hubo un tiempo en que Green Leaves era una canción popular
